In [2]:
import pandas as pd

# Get the data from the model

In [3]:
#TEMPORARY

# Get the historical betting data

In [95]:
bets = pd.read_pickle('../data/pkl/sbr_betting_data.pkl')
bets = bets.reset_index(drop=True)

## Fix the messed up column names from the pickle data

In [99]:
temp = ['Draft_Kings', 'Fanduel', 'Caesars', 'Pointsbet', 'Wynn', 'Betrivers']
merge_df = bets[['Game_Date', 'Team_Name', 'Home', 'Opponent', 'Points', 'Pct_of_Bets',
       'Opening_Spread', 'Opening_Odds', 'Betmgm', 'Betmgm_Odds']]
for i in range(12):
    if i % 2 == 0:
        name = temp[int(i/2)]
        temp_df = pd.DataFrame(bets.iloc[:, (i+10)]).rename(columns={f'{name}_Odds': f'{name}_Spread'}).reset_index(drop=True)
    else:
        name = temp[int(i/2)-1]
        temp_df = pd.DataFrame(bets.iloc[:, (i+10)]).rename(columns={f'{name}_Odds': f'{name}_Odds'}).reset_index(drop=True)
    merge_df = merge_df.merge(temp_df, left_index=True, right_index=True)
bets = merge_df.copy()
bets.rename(columns={'Betmgm': 'Betmgm_Spread'}, inplace=True)

In [101]:
bets.columns

Index(['Game_Date', 'Team_Name', 'Home', 'Opponent', 'Points', 'Pct_of_Bets',
       'Opening_Spread', 'Opening_Odds', 'Betmgm_Spread', 'Betmgm_Odds',
       'Draft_Kings_Spread', 'Draft_Kings_Odds', 'Fanduel_Spread',
       'Fanduel_Odds', 'Caesars_Spread', 'Caesars_Odds', 'Pointsbet_Spread',
       'Pointsbet_Odds', 'Wynn_Spread', 'Wynn_Odds', 'Betrivers_Spread',
       'Betrivers_Odds'],
      dtype='object')

In [102]:
sportsbooks_spreads = ['Betmgm_Spread', 'Draft_Kings_Spread', 'Fanduel_Spread', 
                       'Caesars_Spread', 'Pointsbet_Spread', 'Wynn_Spread', 'Betrivers_Spread']

sportsbooks_spreads_odds = ['Betmgm_Spread', 'Betmgm_Odds',
       'Draft_Kings_Spread', 'Draft_Kings_Odds', 'Fanduel_Spread',
       'Fanduel_Odds', 'Caesars_Spread', 'Caesars_Odds', 'Pointsbet_Spread',
       'Pointsbet_Odds', 'Wynn_Spread', 'Wynn_Odds', 'Betrivers_Spread',
       'Betrivers_Odds']

In [103]:
bets

,Game_Date,Team_Name,Home,Opponent,Points,Pct_of_Bets,Opening_Spread,Opening_Odds,Betmgm_Spread,Betmgm_Odds,...,Fanduel_Spread,Fanduel_Odds,Caesars_Spread,Caesars_Odds,Pointsbet_Spread,Pointsbet_Odds,Wynn_Spread,Wynn_Odds,Betrivers_Spread,Betrivers_Odds
0,2023-03-05,Phoenix,0,Dallas,130,56%,+1.5,-110,-1.5,-115,...,-2,-108,-2,-110,-2,-110,-2,-110,-2,-112
1,2023-03-05,Dallas,1,Phoenix,126,44%,-1.5,-110,+1.5,-105,...,+2,-112,+2,-110,+2,-110,+2,-110,+2,-112
2,2023-03-05,Indiana,0,Chicago,125,53%,+6.5,-110,+5.5,-110,...,+5.5,-108,+5,-110,+5.5,-110,+5.5,-110,+5.5,-110
3,2023-03-05,Chicago,1,Indiana,122,47%,-6.5,-110,-5.5,-110,...,-5.5,-112,-5,-110,-5.5,-110,-5.5,-110,-5.5,-112
4,2023-03-05,Golden State,0,L.A. Lakers,105,60%,-5.5,-110,-5.5,-105,...,-5.5,-118,-5.5,-110,-5,-110,-5,-110,-5,-109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9861,2019-10-05,San Antonio,1,Orlando,89,-,-5.5,-110,,-,...,,-,,-,-6.5,-110,,-,,-
9862,2019-10-04,Houston,0,L.A. Clippers,109,-,-4.5,-110,,-,...,,-,,-,-5,-110,,-,,-
9863,2019-10-04,L.A. Clippers,1,Houston,96,-,+4.5,-110,,-,...,,-,,-,+5,-110,,-,,-
9864,2019-10-04,Indiana,0,Sacramento,132,-,+2.5,-110,,-,...,,-,,-,+5.5,-110,,-,,-


# Compare the data

# Compare the data using selection